# Курсовая работа по СиАОД
## Выполнил студент группы БСТ2205 Шулапов Н. С.

### Задание
Написать программу, которая будет считывать данные из CSV файла, содержащего
информацию о продажах товаров в магазине. Данные в файле содержатся в следующем
формате:
| Номер заказа | Дата заказа | Название товара | Категория товара | Количество продаж | Цена
за единицу | Общая стоимость |
Необходимо:
1. Рассчитать общую выручку магазина.
2. Найти товар, который был продан наибольшее количество раз.
3. Найти товар, который принес наибольшую выручку.
4. Составить отчет, содержащий информацию об общей выручке магазина, количестве
проданных единиц каждого товар и доле каждого товара в общей выручке.

### Выполнение

In [25]:
import csv
import pandas as pd

#### Исходная таблица

In [26]:
df = pd.read_csv("siaod_table.csv", delimiter=';', index_col='Номер заказа')
df

,Дата заказа,Название товара,Категория товара,Количество продаж,Цена за единицу,Общая стоимость
Номер заказа,,,,,,
10001,11.01.2024,Apple iPhone 15 Pro Max,Смартфоны,483,124990,50370170
10002,15.01.2024,Samsung Galaxy S22 Ultra,Наушники,986,20990,20696140
10003,23.01.2024,PlayStation 5 slim,Игровые приставки,541,48990,26503590
10004,30.01.2024,Apple Pencil,Стилусы,67,10990,736330
10005,02.02.2024,Apple MacBook Air 13,Ноутбуки,147,75990,11170530
10006,17.02.2024,Apple iPad Pro 11,Планшеты,99,79990,7919010
10007,18.02.2024,Apple Airpods Max,Наушники,28,52990,1483720
10008,29.02.2024,Infinix ZERO 30,Смартфоны,1921,18999,3649707
10009,09.03.2024,Яндекс ТВ Станция,Телевизоры,246,34999,8609754


Создаётся три класса:

Первый <b>Item</b> - Класс товара в таблице, содержащий все основные атрибуты товара: номер заказа, дату, название, цену и т.д.

Второй <b>Table</b> - Класс, содержащий строки таблицы, то есть класс будет содержать список из класса <b>Item</b>, в нем также будут реализованы некоторые функции для обработки строк таблицы (класса Item)

Третий <b>Report</b> - Класс для формирования отчёта, получает данные из класса <b>Table</b> и выводит общую выручку магазина, товар с наибольшим объемом продаж, и товар, который принес наибольшую прибыль, также выводит сводную таблицу всех товаров с долей в общей прибыли магазина

In [31]:
class Item:

    def __init__(self, attributes: list):
        order_number, order_date, product_name, category, sales_quantity, price_per_unit, total_cost = attributes
        self.order_number = order_number
        self.order_date = order_date
        self.product_name = product_name
        self.category = category
        self.sales_quantity = int(sales_quantity)
        self.price_per_unit = int(price_per_unit)
        self.total_cost = int(total_cost)


In [32]:
class Table:

    def __init__(self, items: list):
        self.items = items
        self.item_get_col = {  
            'order_number': lambda x: x.order_number,
            'order_date': lambda x: x.order_date,
            'product_name': lambda x: x.product_name,
            'category': lambda x: x.category,
            'sales_quantity': lambda x: x.sales_quantity,
            'price_per_unit': lambda x: x.price_per_unit,
            'total_cost': lambda x: x.total_cost
        }

    def get_size(self) -> int:
        return len(self.items)

    def get_column(self, column: str) -> list:
        if column not in self.item_get_col.keys():
            return []
        ans = []
        get_lambda = self.item_get_col[column]
        for item in self.items:
            ans.append(get_lambda(item))
        return ans

    def get_column_sum(self, column) -> float:
        try:
            return sum(self.get_column(column))
        except:
            return None

    def find_maximum(self, column: list) -> float:
        max = 0
        for item in column:
            if item > max:
                max = item
        return max


In [37]:
class Report:

    def __init__(self, table):
        self.table = table

    def _profit(self) -> float:
        return self.table.get_column_sum('total_cost')

    def _sales_num(self) -> int:
        return self.table.get_column_sum('sales_quantity')

    def _percent(self, cost: float, profit: float) -> float:
        return round(cost / profit * 100, 2)

    def _most_freq_item(self) -> tuple[list, int]:
        quantity = self.table.find_maximum(self.table.get_column('sales_quantity'))
        ans = []
        ls = self.table.items
        for item in ls:
            if item.sales_quantity == quantity:
                ans.append(item.product_name)
        return (ans, quantity)

    def _most_prof_item(self) -> tuple[list, float]:
        profit = self.table.find_maximum(self.table.get_column('total_cost'))
        ans = []
        ls = self.table.items
        for item in ls:
            if item.total_cost == profit:
                ans.append(item.product_name)
        return (ans, profit)

    def _make_dataframe(self):
        p = []
        total = self._profit()
        n = self.table.get_column('product_name')
        q = self.table.get_column('sales_quantity')
        t = self.table.get_column('total_cost')
        for cost in t:
            p.append(self._percent(cost, total))

        df = pd.DataFrame({'Количество проданных единиц, шт': q,
                           'Общая сумма продаж, руб': t,
                           'Доля товара от общей выручки, %': p}, index=n)
        display(df.sort_values("Общая сумма продаж, руб", ascending=False))

    def make(self):
        print("Отчёт:")
        print(f'Общая выручка магазина составляет {self._profit()} руб, ' +
              f'всего товаров было продано {self._sales_num()} шт')
        print(
            f'Товар с наибольшим объёмом продаж: {", ".join(self._most_freq_item()[0])}, {self._most_freq_item()[1]} шт')
        print(f'Товар, принёсший наибольшую выручку: {", ".join(self._most_prof_item()[0])}, {self._most_prof_item()[1]} руб')
        self._make_dataframe()


In [38]:
with open("siaod_table.csv", mode="r", encoding="utf-8") as f:
    reader = csv.reader(f, delimiter=";")
    header = next(reader) 
    items = []
    for row in reader:
        items.append(Item(row))
        
products_table = Table(items)
report = Report(products_table)
report.make()

Отчёт:
Общая выручка магазина составляет 327389107 руб, всего товаров было продано 11618 шт
Товар(ы) с наибольшим объёмом продаж: Apple iPhone 11, 2004 шт
Товар(ы), принёсшие наибольшую выручку: Apple iPhone 11, 84147960 руб


,"Количество проданных единиц, шт","Общая сумма продаж, руб","Доля товара от общей выручки, %"
Apple iPhone 11,2004,84147960,25.70
Xiaomi 13T,969,53285310,16.28
Apple iPhone 15 Pro Max,483,50370170,15.39
Samsung A54,1987,43694130,13.35
PlayStation 5 slim,541,26503590,8.10
Samsung Galaxy S22 Ultra,986,20696140,6.32
Apple MacBook Air 13,147,11170530,3.41
Яндекс ТВ Станция,246,8609754,2.63
Apple iPad Pro 11,99,7919010,2.42
Samsung Odyssey OLED G9,27,3914730,1.20


### Заключение

Была написана программа, которая считывает данные из CSV файла, содержащего информацию о продажах товаров в магазине, был составлен отчёт: рассчитана общая выручка магазина, были найдены товары с наибольшим числом продаж и товар с наибольшей выручкой, с помощью библиотеки Pandas была составлена таблица, в которой представлены товары магазина, отсортированные по доле в общей выручке магазина